# Aufgabe 1 (Preliminaries)

This task requires a working installation of SQLite. The way to install it depends on your platform, but it is usually very easy. For Ubuntu 20.04 it’s just `sudo apt install sqlite3`

In [1]:
%load_ext sql

# Aufgabe 2 (University Schema)

Laden Sie die gegebene SQLite-Datenbank `university.db`

In [2]:
%sql sqlite:///university.db

1. Schauen Sie sich das Schema an:

        `.tables`

        `.schema --indent`

2. Schauen Sie sich die Instanz an (die Daten in jeder Tabelle):

    `select * from <table-name>;`

3. Vergleichen Sie mit dem in der Vorlesung dazu gegebenen Schema. Verstehen Sie die Bedeutung der Datensätze in allen diesen Tabellen, die Primary Keys und die Foreign-Key-Beziehungen zwischen den Tabellen. Beantworten Sie insbesondere folgende Fragen:

    (a)

    (b)

    (c)